In [1]:
import numpy as np
import pandas as pd

[days][person][genotype]  
id,min,max (hours)

In [7]:
staff_planning = [[[j,0,10] for j in range(11)] for i in range(5)]

In [9]:
staff_planning[:2]

[[[0, 0, 10],
  [1, 0, 10],
  [2, 0, 10],
  [3, 0, 10],
  [4, 0, 10],
  [5, 0, 10],
  [6, 0, 10],
  [7, 0, 10],
  [8, 0, 10],
  [9, 0, 10],
  [10, 0, 10]],
 [[0, 0, 10],
  [1, 0, 10],
  [2, 0, 10],
  [3, 0, 10],
  [4, 0, 10],
  [5, 0, 10],
  [6, 0, 10],
  [7, 0, 10],
  [8, 0, 10],
  [9, 0, 10],
  [10, 0, 10]]]

In [3]:
hourlystaff_needed = np.array([
    [0, 0, 0, 0, 0, 0, 4, 4, 4, 2, 2, 2, 6, 6, 2, 2, 2, 6, 6, 6, 2, 2, 2, 2],
    [0, 0, 0, 0, 0, 0, 4, 4, 4, 2, 2, 2, 6, 6, 2, 2, 2, 6, 6, 6, 2, 2, 2, 2],
    [0, 0, 0, 0, 0, 0, 4, 4, 4, 2, 2, 2, 6, 6, 2, 2, 2, 6, 6, 6, 2, 2, 2, 2],
    [0, 0, 0, 0, 0, 0, 4, 4, 4, 2, 2, 2, 6, 6, 2, 2, 2, 6, 6, 6, 2, 2, 2, 2],
    [0, 0, 0, 0, 0, 0, 4, 4, 4, 2, 2, 2, 6, 6, 2, 2, 2, 6, 6, 6, 2, 2, 2, 2]
])

In [4]:
def employee_present(employee, time):
    employee_start_time = employee[1]
    employee_duration = employee[2]
    employee_end_time = employee_start_time + employee_duration
    if (time >= employee_start_time) and (time < employee_end_time):
        return True
    return False

In [10]:
def staffplanning_to_hourlyplanning(staff_planning):
    hourlystaff_week = []
    for day in staff_planning:
        hourlystaff_day = [] 
        for employee in day:  
            employee_present_hour = []
            for time in range(0, 24):
                employee_present_hour.append(employee_present(employee, time))   
            hourlystaff_day.append(employee_present_hour)  
        hourlystaff_week.append(hourlystaff_day)
    hourlystaff_week = np.array(hourlystaff_week).sum(axis = 1)  
    return hourlystaff_week

In [11]:
def cost(hourlystaff, hourlystaff_needed):
    errors = hourlystaff - hourlystaff_needed
    overstaff = abs(errors[errors > 0].sum())
    understaff = abs(errors[errors < 0].sum())
    
    overstaff_cost = 1
    understaff_cost = 1
    
    cost = overstaff_cost * overstaff + understaff_cost * understaff
    return cost

In [12]:
def generate_random_staff_planning(n_days, n_staff):
    period_planning = []
    for day in range(n_days):
        day_planning = []
        for employee_id in range(n_staff):
            start_time = np.random.randint(0, 23)
            duration = np.random.randint(0, 10)
            employee = [employee_id, start_time, duration]
            day_planning.append(employee)
        period_planning.append(day_planning)
    return period_planning

In [14]:
random_staff_planning = generate_random_staff_planning(n_days = 5, n_staff = 11)
random_staff_planning[:2]

[[[0, 11, 6],
  [1, 13, 9],
  [2, 10, 9],
  [3, 2, 3],
  [4, 0, 1],
  [5, 10, 3],
  [6, 18, 0],
  [7, 12, 1],
  [8, 17, 2],
  [9, 21, 7],
  [10, 2, 7]],
 [[0, 22, 6],
  [1, 4, 6],
  [2, 8, 7],
  [3, 6, 6],
  [4, 5, 5],
  [5, 21, 0],
  [6, 14, 8],
  [7, 7, 4],
  [8, 13, 8],
  [9, 7, 3],
  [10, 19, 1]]]

In [15]:
cost(staffplanning_to_hourlyplanning(random_staff_planning), hourlystaff_needed)

206

In [17]:
def create_parent_generation(n_parents, n_days = 7, n_staff = 11):
    parents = []
    for i in range(n_parents):
        parent = generate_random_staff_planning(n_days = n_days, n_staff = n_staff)
        parents.append(parent)
    return parents

In [18]:
def random_combine(parents, n_offspring):
    n_parents = len(parents)
    n_periods = len(parents[0])
    n_employees = len(parents[0][0])
    
    offspring = []
    for i in range(n_offspring):
        random_dad = parents[np.random.randint(low = 0, high = n_parents - 1)]
        random_mom = parents[np.random.randint(low = 0, high = n_parents - 1)]
        
        dad_mask = np.random.randint(0, 2, size = np.array(random_dad).shape)
        mom_mask = np.logical_not(dad_mask)
        
        child = np.add(np.multiply(random_dad, dad_mask), np.multiply(random_mom, mom_mask))

        offspring.append(child)
    return offspring

In [20]:
def mutate_parent(parent, n_mutations):
    size1 = parent.shape[0]
    size2 = parent.shape[1]
    for i in range(n_mutations):
        rand1 = np.random.randint(0, size1)
        rand2 = np.random.randint(0, size2)
        rand3 = np.random.randint(1, 2)
        parent[rand1,rand2,rand3] = np.random.randint(0, 10)
    return parent

In [21]:
def mutate_gen(parent_gen, n_mutations):
    mutated_parent_gen = []
    for parent in parent_gen:
        mutated_parent_gen.append(mutate_parent(parent, n_mutations))
    return mutated_parent_gen

In [22]:
def is_acceptable(parent):
    return np.logical_not((np.array(parent)[:,:,2:] > 10).any())

In [23]:
def select_acceptable(parent_gen):
    parent_gen = [parent for parent in parent_gen if is_acceptable(parent)]
    return parent_gen

In [24]:
def select_best(parent_gen, hourlystaff_needed, n_best):
    costs = []
    for idx, parent_staff_planning in enumerate(parent_gen):
        
        parent_hourly_planning = staffplanning_to_hourlyplanning(parent_staff_planning)
        parent_cost = cost(parent_hourly_planning, hourlystaff_needed)
        costs.append([idx, parent_cost])
    
    print('generations best is: {}, generations worst is: {}'.format(pd.DataFrame(costs)[1].min(), pd.DataFrame(costs)[1].max()))
    
    costs_tmp = pd.DataFrame(costs).sort_values(by = 1, ascending = True).reset_index(drop=True)
    selected_parents_idx = list(costs_tmp.iloc[:n_best,0])
    selected_parents = [parent for idx, parent in enumerate(parent_gen) if idx in selected_parents_idx]
    
    return selected_parents

In [26]:
def gen_algo(hourlystaff_needed, n_iterations):   
    
    generation_size = 5000
    
    parent_gen = create_parent_generation(n_parents = generation_size, n_days = 5, n_staff = 11)
    for it in range(n_iterations):
        parent_gen = select_acceptable(parent_gen)
        parent_gen = select_best(parent_gen, hourlystaff_needed, n_best = 100)
        parent_gen = random_combine(parent_gen, n_offspring = generation_size)
        parent_gen = mutate_gen(parent_gen, n_mutations = 1)
    
    best_child = select_best(parent_gen, hourlystaff_needed, n_best = 1)
    return best_child

In [27]:
best_planning = gen_algo(hourlystaff_needed, n_iterations = 100)


generations best is: 164, generations worst is: 287
generations best is: 144, generations worst is: 260
generations best is: 138, generations worst is: 224
generations best is: 124, generations worst is: 214
generations best is: 120, generations worst is: 204
generations best is: 113, generations worst is: 203
generations best is: 102, generations worst is: 191
generations best is: 95, generations worst is: 182
generations best is: 92, generations worst is: 181
generations best is: 89, generations worst is: 181
generations best is: 82, generations worst is: 173
generations best is: 80, generations worst is: 179
generations best is: 71, generations worst is: 182
generations best is: 72, generations worst is: 169
generations best is: 70, generations worst is: 149
generations best is: 61, generations worst is: 149
generations best is: 61, generations worst is: 143
generations best is: 58, generations worst is: 138
generations best is: 57, generations worst is: 137
generations best is: 54,

In [28]:
print(best_planning)

[array([[[ 0, 17,  3],
        [ 1, 12,  8],
        [ 2,  6,  8],
        [ 3,  6,  8],
        [ 4,  6,  4],
        [ 5, 17,  7],
        [ 6, 12,  2],
        [ 7,  7,  2],
        [ 8, 12,  8],
        [ 9, 17,  9],
        [10, 16,  4]],

       [[ 0, 12,  8],
        [ 1,  6,  2],
        [ 2, 17,  8],
        [ 3,  7,  7],
        [ 4, 12,  8],
        [ 5, 17,  3],
        [ 6, 12,  8],
        [ 7,  6,  3],
        [ 8, 12,  2],
        [ 9, 17,  9],
        [10,  6,  8]],

       [[ 0, 12,  8],
        [ 1,  6,  8],
        [ 2,  6,  8],
        [ 3,  6,  3],
        [ 4, 12,  8],
        [ 5, 17,  2],
        [ 6, 12,  8],
        [ 7, 17,  9],
        [ 8, 12,  1],
        [ 9, 17,  7],
        [10,  6,  3]],

       [[ 0, 17,  7],
        [ 1, 17,  8],
        [ 2,  7,  7],
        [ 3, 17,  3],
        [ 4, 12,  8],
        [ 5,  6,  4],
        [ 6, 12,  2],
        [ 7, 17,  3],
        [ 8,  6,  8],
        [ 9,  6,  3],
        [10, 12,  8]],

       [[ 0,  6,  3],
 

In [29]:
print(staffplanning_to_hourlyplanning(best_planning[0]))

[[0 0 0 0 0 0 3 4 4 3 2 2 5 5 2 2 3 6 6 6 2 2 2 2]
 [0 0 0 0 0 0 3 4 3 2 2 2 6 6 3 3 3 6 6 6 2 2 2 2]
 [0 0 0 0 0 0 4 4 4 2 2 2 6 5 3 3 3 6 6 5 2 2 2 2]
 [0 0 0 0 0 0 3 4 4 3 2 2 5 5 2 2 2 6 6 6 2 2 2 2]
 [0 0 0 0 0 0 4 4 4 2 2 3 5 5 2 2 2 6 6 6 3 2 2 2]]
